In [22]:
from py2neo import Graph
from pandas import DataFrame

In [23]:
graph = Graph(host="localhost://7474", auth=("neo4j", "290287"))

In [48]:
a=graph.run("MATCH (a:Person) RETURN a.name, a.born LIMIT 4").data()
a

[{'a.name': 'Keanu Reeves', 'a.born': 1964},
 {'a.name': 'Carrie-Anne Moss', 'a.born': 1967},
 {'a.name': 'Laurence Fishburne', 'a.born': 1961},
 {'a.name': 'Hugo Weaving', 'a.born': 1960}]

# Our graph backend

In [112]:
def search_restaurant_by_food(text):
    result=[]
    key_words = text
    query = "MATCH (n:`Restaurant`)-[:`Serves`]->(f:`Food` {name:'%s'})  RETURN n.name, n.sentiment_score ORDER BY n.sentiment_score DESC LIMIT 10" % key_words
    result = graph.run(query).data()
    if result != []:
        return result
    else:
        key_words = key_words.split()
        for i in range(len(key_words)):
            query = "MATCH (n:`Restaurant`)-[:`Serves`]->(f:`Food` {name:'%s'})  RETURN n.name, n.sentiment_score ORDER BY n.sentiment_score DESC LIMIT 10" % key_words[i]
            temp = graph.run(query).data()
            result.append(temp)
        if result == [[]]:
            return ['No records found']
        else:
            return result
        

def search_restaurant_by_cusine(text):
    result=[]
    key_words = text
    query = "MATCH (n:`Restaurant` {cusine:'%s'})  RETURN n.name, n.sentiment_score ORDER BY n.sentiment_score DESC LIMIT 10" % key_words
    result = graph.run(query).data()
    if result == []:
        return ['No records found']
    else: 
        return result
    

def find_reviews_by_restaurant_name(name):
    result = []
    key_words = name
    query = "MATCH (n:`Restaurant` {name:'%s'})<-[:`ABOUT`]-(r:`Review`)  RETURN n.name, r.content ORDER BY r.like DESC LIMIT 3" % key_words
    result = graph.run(query).data()
    if result == []:
        return ['No reviews']
    else: 
        return result

# Testing on Movie graph

In [124]:

def search_movie_by_actor(text):
    result=[]
    key_words = text
    query = "MATCH (p:`Person` {name:'%s'})-[:`ACTED_IN`]->(m:`Movie`)  RETURN m.title, m.released ORDER BY m.released DESC LIMIT 10" % key_words
    result = graph.run(query).data()
    if result != []:
        print(1)
        return result
    else:
        key_words = key_words.split()
        for i in range(len(key_words)):
            query = "MATCH (p:`Person` {name:'%s'})-[:`ACTED_IN`]->(m:`Movie`)  RETURN m.title, m.released ORDER BY m.released DESC LIMIT 10" % key_words[i]
            temp = graph.run(query).data()
            result.append(temp)
        if result == [[]]:
            print(2)
            return ['No records found']
        else:
            print(3)
            return result
      
    
def find_reviews_by_movie_name(name):
    result = []
    key_words = name
    query = 'MATCH (m:`Movie` {title:"%s"})<-[r:`REVIEWED`]-()  RETURN r.summary LIMIT 3' % key_words
    result = graph.run(query).data()
    if result == []:
        return ['No reviews']
    else: 
        return result

    
def reviews_result(name):
    reviews = []
    for item in name:
        temp = DataFrame(find_reviews_by_movie_name(item))
        temp = temp['r.summary'].tolist()
        dic = {item:temp}
        reviews.append(dic)
    return reviews

In [125]:
b=search_movie_by_actor('Keanu Reeves')

1


In [126]:
b

[{'m.title': 'The Matrix Revolutions', 'm.released': 2003},
 {'m.title': 'The Matrix Reloaded', 'm.released': 2003},
 {'m.title': "Something's Gotta Give", 'm.released': 2003},
 {'m.title': 'The Replacements', 'm.released': 2000},
 {'m.title': 'The Matrix', 'm.released': 1999},
 {'m.title': "The Devil's Advocate", 'm.released': 1997},
 {'m.title': 'Johnny Mnemonic', 'm.released': 1995}]

In [136]:
#find reviews
name =['The Replacements','The Birdcage','Unforgiven']
reviews = []
for item in name:
    temp = DataFrame(find_reviews_by_movie_name(item))
    temp = temp['r.summary'].tolist()
    dic = {item:temp}
    reviews.append(dic)

In [137]:
reviews

[{'The Replacements': ['The coolest football movie ever',
   'Pretty funny at times',
   'Silly, but fun']},
 {'The Birdcage': ["Slapstick redeemed only by the Robin Williams and Gene Hackman's stellar performances"]},
 {'Unforgiven': ['Dark, but compelling']}]

['The coolest football movie ever', 'Pretty funny at times', 'Silly, but fun']